In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROOQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.1-8b-instant",temperature=1.9)

result=llm.invoke("Howdy")
result

AIMessage(content="Howdy back at ya! What's brewin'?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 37, 'total_tokens': 49, 'completion_time': 0.01635713, 'prompt_time': 0.002331444, 'queue_time': 0.216946143, 'total_time': 0.018688574}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_90c2e79dab', 'finish_reason': 'stop', 'logprobs': None}, id='run--ddeb5f23-5443-4de6-a324-6f4fa01e0f68-0', usage_metadata={'input_tokens': 37, 'output_tokens': 12, 'total_tokens': 49})

In [2]:
from typing import Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class State(TypedDict):
    joke : str
    topic: str
    feedback: str
    funny_or_not:str

class Feedback(BaseModel):
    grade: Literal["funny", "not funny"]= Field(description="Decide if the joke is funny or not.")
    feedback: str= Field(description="If the joke is not funny,")
evaluator=llm.with_structured_output(Feedback)

In [3]:
def generator_llm(state:State):
    """ LLM generates a joke"""    
    if(state.get("feedback")):
        generator=llm.invoke(
            f"Write a joke about {state['topic']} but take into account the feedback {state['feedback']}"
            )
    else:
        generator=llm.invoke(
            f"Write a joke about {state['topic']}"
            )
    print(generator.content)
    return {"joke": generator.content}

def evaluator_llm(state:State):
    """ LLM evaluates a joke"""    
    grade=evaluator.invoke(
        f"Grade the joke {state['joke']}"
    )
    print(grade)
    return {"funny_or_not":grade.grade, "feedback":grade.feedback}

def route_joke(state:State):
    """ Routes back to the generator or end from evaluator"""  
   
    if(state["funny_or_not"]=="funny"):
        print("Accepted")
        return "Accepted"
    if(state["funny_or_not"]=="not funny"):
        print("Rejected")
        return "Rejected + Feedback"
    
    

In [4]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END

graph= StateGraph(State)

#Add nodes
graph.add_node("generator_llm",generator_llm)
graph.add_node("evaluator_llm",evaluator_llm)

#Add edges
graph.add_edge(START,"generator_llm")
graph.add_edge("generator_llm","evaluator_llm")
graph.add_conditional_edges(
    "evaluator_llm",
    route_joke,
    {
        "Accepted":END,
        "Rejected + Feedback":"generator_llm"
    }
)

#Compile graph
graph=graph.compile()

# graph_image=graph.get_graph().draw_mermaid_png()
# display(Image(graph_image))

In [5]:
state=graph.invoke({"topic":"Write a single line joke"})
print(state['joke'])

Why did the cat join a band? Because it wanted to be a purr-cussionist.
grade='not funny' feedback='The cat in band as purr-cussionis was quite corny but did manage to pull a very faint grin from this side; perhaps some will like it. '
Rejected
Why did the musician always bring a ladder to the gig? He wanted to take his solos to new heights. "New heights" sounds similar to what one says of musical talent being 'at new hights, however this was written 'at it and not as that'
grade='not funny' feedback="The joke sounds like wordplay related more to common sayings 'heights' but not much to do with the 'gig' so this makes the punch line confusing."
Rejected
A musician goes to a new high-security venue - they charge high fees so now I need my Gig with Heights.

However, here is the joke in two simple lines, to improve the connection:

A musician goes to gig at the Eiffel Tower 
I guess I needed that heights.
grade='not funny' feedback="You need more connection between this line 'Eiffel TOWE